In [39]:
import numpy as np
import pandas as pd
import re
import string

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [40]:
true = pd.read_csv('True.csv')
false = pd.read_csv('Fake.csv')

In [41]:
true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [42]:
false.head(5)


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [43]:
true["label"] = 1
false["label"] = 0

In [44]:
false.head(5)
true.drop(["text", "subject", "date"], axis=1, inplace=True)
true.head(5)
false.drop(["text", "subject", "date"], axis=1, inplace=True)
false.head(5)

,title,label
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0


In [45]:
dataset = pd.concat([true, false], axis=0, ignore_index=True)

In [46]:
dataset.head(5)

,title,label
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [47]:
dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)
dataset.head(5)

,title,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,0
1,Failed GOP Candidates Remembered In Hilarious...,0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,0
3,California AG pledges to defend birth control ...,1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,0


In [48]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, )
    text = re.sub(r'\@\w+|\#', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub("\n" ,"" , text)

    return text

In [49]:
dataset['title'] = dataset['title'].apply(clean_text)

In [50]:
vectorizer = TfidfVectorizer()

In [51]:
from sklearn.model_selection import cross_val_score

In [52]:
# x = np.array('title').reshape(-1, 1)
# x = vectorizer.fit_transform(dataset['title'])
# y = dataset['label']
# # scores = cross_val_score(lr, x, y, cv=5)
# print("Cross-validation Accuracy:", scores.mean())

In [53]:
# lr = LogisticRegression()
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# lr.fit(x_train, y_train)
# y_pred = lr.predict(x_test)

In [54]:
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy * 100:.2f}%')

In [55]:
# print(classification_report(y_test, y_pred))

In [56]:
# joblib.dump(lr, 'model95.jb')
# joblib.dump(vectorizer , 'vectorizer95.jb')

Training Model using BERT

In [57]:
!pip install transformers datasets torch

In [63]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    dataset['title'].tolist(),
    dataset['label'].tolist(),
    test_size=0.2,
    random_state=42
)

In [65]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=64)

In [66]:
import torch

class FakeNewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = FakeNewsDataset(train_encodings, train_labels)
val_dataset = FakeNewsDataset(val_encodings, val_labels)


load bert

In [68]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trarining

In [70]:
!pip install --upgrade transformers

In [71]:
from transformers import Trainer, TrainingArguments

In [74]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    logging_dir='./logs',
    logging_steps=10,
)


In [76]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

print accuracy

In [77]:
preds = trainer.predict(val_dataset)
y_pred = preds.predictions.argmax(-1)

from sklearn.metrics import accuracy_score, classification_report
print("Accuracy:", accuracy_score(val_labels, y_pred))
print(classification_report(val_labels, y_pred))

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pradeepapex27 (pradeepapex27-national-institute-of-technology-tiruchira) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Accuracy: 0.5199331848552339
              precision    recall  f1-score   support

           0       0.52      1.00      0.68      4669
           1       0.00      0.00      0.00      4311

    accuracy                           0.52      8980
   macro avg       0.26      0.50      0.34      8980
weighted avg       0.27      0.52      0.36      8980



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


save model


In [78]:
model.save_pretrained("./bert-fake-news-model")
tokenizer.save_pretrained("./bert-fake-news-model")


('./bert-fake-news-model/tokenizer_config.json',
 './bert-fake-news-model/special_tokens_map.json',
 './bert-fake-news-model/vocab.txt',
 './bert-fake-news-model/added_tokens.json')